<a href="https://colab.research.google.com/github/dougc333/AbsoluteSWDD/blob/master/flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
!pip install prometheus-client

In [27]:


!pip install flask-ngrok
from flask_ngrok import run_with_ngrok

In [28]:
#https://colab.research.google.com/github/srivatsan88/YouTubeLI/blob/master/Running_Flask_in_Colab.ipynb#scrollTo=xk6pqXHXpChi
import socket
print(socket.gethostbyname(socket.gethostname()))
from flask_ngrok import run_with_ngrok

172.28.0.12


In [30]:
from flask import Flask, jsonify, make_response, request
import threading


app = Flask(__name__)

@app.route("/")
def hello():
    return "I am alive!!!!!!!!!!!!!!!!!!!"

#run_with_ngrok(app)
#app.run()
#
threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':6000}).start()

 * Serving Flask app '__main__'
 * Debug mode: off


In [32]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://5000-gpu-t4-s-1z72eomj6xnaa-a.us-west4-0.prod.colab.dev


In [31]:
import requests
res = requests.get("http://172.28.0.12:6000/")
print(res.status_code)
print("res:",req)
print("********************************")
print(dir(res))
print(f"res content:{res.content}")
print(res.text) #response is intercepted

200
res: <Response [200]>
********************************
['__attrs__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_content', '_content_consumed', '_next', 'apparent_encoding', 'close', 'connection', 'content', 'cookies', 'elapsed', 'encoding', 'headers', 'history', 'is_permanent_redirect', 'is_redirect', 'iter_content', 'iter_lines', 'json', 'links', 'next', 'ok', 'raise_for_status', 'raw', 'reason', 'request', 'status_code', 'text', 'url']
res content:b'<!DOCTYPE HTML>\n<html>\n\n<head>\n    <meta charset="utf-8">\n\n    <title>Home Page - Select or create a notebook</title>\n    <link id="fa

In [33]:
def create_app(test_config=None):
    "Create and configure an instance of the Flask application."
    app = Flask(__name__, instance_relative_config=True)
    app.config.from_mapping(
        # a default secret that should be overridden by instance config
        SECRET_KEY="dev"
    )

    if test_config is None:
        # load the instance config, if it exists, when not testing
        app.config.from_pyfile("config.py", silent=True)
    else:
        # load the test config if passed in
        app.config.update(test_config)

    @app.route("/hello")
    def hello_colab():
        return f"Hello, Colab!"

    @app.route("/hello/<human_name>")
    def hello(human_name):
        return f"Hello, {human_name}!"

    app.add_url_rule("/", endpoint="hello_colab")

    return app

In [34]:
assert not create_app().testing
assert create_app({"TESTING": True}).testing

In [35]:
app = create_app({"TESTING": True})
client = app.test_client()
response = client.get("/hello/Pieter van de Heuvel, MSc")
print(response.text)
assert response.data == b"Hello, Pieter van de Heuvel, MSc!"
response = client.get("/")
print(response.data)
assert response.data == b"Hello, Colab!"

Hello, Pieter van de Heuvel, MSc!
b'Hello, Colab!'


In [ ]:
from prometheus_client import CollectorRegistry, Gauge, push_to_gateway

registry = CollectorRegistry()
g = Gauge('model_training_loss', 'Loss value', registry=registry)
g.set(0.123)

# Push to external Prometheus push gateway (must be set up externally)
push_to_gateway('YOUR_SERVER_IP:9091', job='colab-training', registry=registry)

In [ ]:
from prometheus_client import CollectorRegistry, Gauge, push_to_gateway

registry = CollectorRegistry()
g = Gauge('model_training_loss', 'Loss value', registry=registry)
g.set(0.123)

# Push to external Prometheus push gateway (must be set up externally)
push_to_gateway('YOUR_SERVER_IP:9091', job='colab-training', registry=registry)

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [7]:
#MNIST/raw
# 4 files
#train-images-idx3-ubyte.gz: training set images (9912422 bytes)
#train-labels-idx1-ubyte.gz: training set labels (28881 bytes)
#t10k-images-idx3-ubyte.gz: test set images (1648877 bytes)
#t10k-labels-idx1-ubyte.gz: test set labels (4542 bytes)

#is the normalization centering?
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = torchvision.datasets.MNIST(root='.', train=True, download=False, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='.', train=False, download=False, transform=transform)


In [9]:
print(train_dataset)
print(len(train_dataset))
print(test_dataset)
print(len(test_dataset))


Dataset MNIST
    Number of datapoints: 60000
    Root location: .
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )
60000
Dataset MNIST
    Number of datapoints: 10000
    Root location: .
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )
10000


In [ ]:
# Define the image classifier model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3),
            nn.ReLU()
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 22 * 22, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x